**Importing Libraries**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
!pip install pycaret
from pycaret.time_series import *

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.9/485.9 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 46.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.5/160.5 kB 13.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.0/258.0 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.7/80.7 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.9/21.9 MB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 29.0 MB

ImportError: cannot import name '_format_load_msg' from 'joblib.memory' (/usr/local/lib/python3.10/dist-packages/joblib/memory.py)

**Mounting the Google Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

**Importing the dataset**

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/BTC_CANDLESTICK_DATA(19 lakh rows).csv',index_col=0)
df

**Data Preparation**

In [ ]:
del df["symbol"]
del df["token"]
del df["hour"]
del df["day"]
columns_to_drop = list(range(7, 22))  # List of column indices to drop
df.drop(df.columns[columns_to_drop], axis=1, inplace=True)


In [ ]:
df

In [ ]:
del df["Volume USDT"]
del df["tradecount"]
del df["ema_5"]

In [ ]:
df

In [ ]:
df["next_close"] = df["close"].shift(-1)
df

In [ ]:
df = df.iloc[:-1]

**Finding the best algorithm using pycaret**

In [ ]:
from pycaret.regression import *
exp_reg = setup(data=df, target='next_close', train_size=0.8, session_id=110)
best_model = compare_models(exclude=['rf','huber','lar','omp','dummy','ada','et','gbr'])
evaluate_model(best_model)

**Training model**

In [ ]:
training_df = df.iloc[:-100]

**On running pycaret on smaller section of the dataset lasso least angle regression showed less mse so i have also trained one model using it**

In [ ]:
from sklearn.linear_model import LassoLars
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split


X = training_df[['open', 'high', 'low', 'close']]
y = training_df['next_close']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LassoLars(alpha=0.1)

# Training the model
model.fit(X_train, y_train)

# Making predictions
y_pred = model.predict(X_test)

# Evaluating the model
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)


**Manually trying model **

In [ ]:
model.predict([[29964.99, 29969.99, 29964.99, 29967.58]])

In [ ]:
model.predict([[29967.59, 29973.41, 29967.58, 29973.41]])

In [ ]:
model.predict([[29973.40, 29976.80, 29973.40, 29976.80]])

In [ ]:
model.predict([[29976.79, 29985.00, 29976.79, 29984.99]])

**Training on linear regression model**

In [ ]:

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

X = training_df[['open', 'high', 'low', 'close']]
y = training_df['next_close']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the linear regression model
model2 = LinearRegression()
model2.fit(X_train, y_train)

# Make predictions
y_pred = model2.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

In [ ]:
model2.predict([[29964.99, 29969.99, 29964.99, 29967.58]])

**Training on BayesianRidg model**

In [ ]:
from sklearn.linear_model import BayesianRidge
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

X = training_df[['open', 'high', 'low', 'close']]
y = training_df['next_close']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Instantiate the Bayesian Ridge model
model3 = BayesianRidge()

# Training the model
model3.fit(X_train, y_train)

# Making predictions
y_pred = model3.predict(X_test)

# Evaluating the model
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)


In [ ]:
model3.predict([[29964.99, 29969.99, 29964.99, 29967.58]])